In [1]:
import requests
import pandas as pd
import numpy as np
import os
from google.cloud import storage


def fetch_data(team_id, league_id, season_year):
    url = "https://api-football-v1.p.rapidapi.com/v3/players"
    querystring = {"team": team_id, "league": league_id, "season": season_year}
    headers = {
        "X-RapidAPI-Key": "YOUR_API_KEY",
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    return response.json()

def process_data(data):
    players_data = []
    for player in data['response']:
        player_info = player['player']
        stats = player['statistics'][0]
        player_data = {
            "Player name": player_info['name'],
            "Nationality": player_info['nationality'],
            "Position": player_info.get('position', 'N/A'),
            "Weight": player_info.get('weight', 'N/A'),
            "Height": player_info.get('height', 'N/A'),
            "Team": stats['team']['name'],
            "League": stats['league']['name'],
            "Appearances": stats['games']['appearences'],
            "Lineups": stats['games']['lineups'],
            "Minutes": stats['games']['minutes'],
            "Goals": stats['goals']['total'],
            "Assists": stats['goals']['assists'],
            "Yellow Cards": stats['cards']['yellow'],
            "Red Cards": stats['cards']['red'],
            "Shots Total": stats['shots']['total'],
            "Shots On": stats['shots']['on'],
            "Passes Total": stats['passes']['total'],
            "Key Passes": stats['passes']['key'],
            "Pass Accuracy": stats['passes']['accuracy'],
            "Tackles Total": stats['tackles']['total'],
            "Blocks": stats['tackles']['blocks'],
            "Interceptions": stats['tackles']['interceptions'],
            "Dribbles Attempts": stats['dribbles']['attempts'],
            "Dribbles Success": stats['dribbles']['success'],
            "Dribbled Past": stats['dribbles']['past'],
            "Fouls Drawn": stats['fouls']['drawn'],
            "Fouls Committed": stats['fouls']['committed'],
            "Duels Total": stats['duels']['total'],
            "Duels Won": stats['duels']['won'],
            "Penalty Scored": stats['penalty']['scored'],
            "Penalty Missed": stats['penalty']['missed'],
            "Penalty Saved": stats['penalty']['saved']
        }
        players_data.append(player_data)
    return pd.DataFrame(players_data)

def clean_and_transform_data(df):
    # Drop unnecessary columns
    df = df.drop(['Position', 'Dribbled Past'], axis=1)

    # Cleaning Weight column
    # Remove 'kg' and convert 'none' to NaN, then convert to numeric
    df['Weight'] = df['Weight'].str.replace(' kg', '').str.replace('kg', '')
    df['Weight'].replace('none', np.nan, inplace=True)
    df['Weight'] = pd.to_numeric(df['Weight'], errors='coerce')

    # Calculate the average weight and replace NaN values with it
    average_weight = df['Weight'].dropna().mean()
    df['Weight'].fillna(average_weight, inplace=True)

    # Cleaning Height column
    # Remove 'cm' and convert 'none' to NaN, then convert to numeric
    df['Height'] = df['Height'].str.replace(' cm', '').str.replace('cm', '')
    df['Height'].replace('none', np.nan, inplace=True)
    df['Height'] = pd.to_numeric(df['Height'], errors='coerce')

    # Calculate the average height and replace NaN values with it
    average_height = df['Height'].dropna().mean()
    df['Height'].fillna(average_height, inplace=True)

    # Replace all other NaN values in the DataFrame with 0
    df.fillna(0, inplace=True)

    return df

def save_to_local(df, local_folder, file_name):
    # Create the directory if it doesn't exist
    if not os.path.exists(local_folder):
        os.makedirs(local_folder)

    # Define the file path
    file_path = os.path.join(local_folder, file_name)

    # Save DataFrame to a CSV locally without header and index
    df.to_csv(file_path, index=False, header=False)
    print(f"File {file_path} saved locally.")

def main():
    # Prompt user for input
    team_id = input("Enter team ID: ")
    league_id = input("Enter league ID: ")
    season_year = input("Enter season year: ")

    # Fetch, process, and clean data
    data = fetch_data(team_id, league_id, season_year)
    df = process_data(data)
    df_cleaned = clean_and_transform_data(df)

    # Create file name based on input and save to local
    file_name = f"players_data_team_{team_id}_league_{league_id}_season_{season_year}.csv"
    save_to_local(df_cleaned, "local_exported_csv", file_name)

if __name__ == "__main__":
    main()


KeyError: 'response'